# Sprint12 CNN2

データセットの用意

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from numpy import linalg as LA
import copy
sns.set()
%matplotlib inline
import time
import math
import copy
import random

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [109]:
# MNIST data set
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [110]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
print(y_train.shape) # (60000,)
print(y_train_one_hot.shape) # (60000, 10)
print(y_train_one_hot.dtype) # float64

(60000,)
(60000, 10)
float64


In [111]:
X_train, X_val, y_train_one_hot, y_val_one_hot = train_test_split(X_train, y_train_one_hot, test_size=0.2)
print(X_train.shape) # (48000, 28, 28)
print(X_val.shape) # (12000, 28, 28)

'''
今回は白黒画像ですからチャンネルは1つしかありませんが、
チャンネル方向の軸は用意しておく必要があります。
(n_samples, n_channels, height, width)のNCHW
または(n_samples, height, width, n_channels)のNHWC どちらかの形にしてください。
'''
#(n_samples, n_channels, height, width)のNCHW
X_train = X_train.reshape(48000, 1, 28, 28)
X_val = X_val.reshape(12000, 1, 28, 28)

print(X_train.shape) # (48000, 1, 28, 28)
print(X_val.shape) # (12000, 1, 28, 28)

(48000, 28, 28)
(12000, 28, 28)
(48000, 1, 28, 28)
(12000, 1, 28, 28)


## 【問題1】2次元畳み込み層の作成
>1次元畳み込み層のクラスConv1dを発展させ、2次元畳み込み層のクラスConv2dを作成してください。

In [226]:
class hiraki:
    def __init__(self, F_size_h, F_size_w, 
                 stride = 1, padding = 0):
        self.F_size_h = F_size_h
        self.F_size_w = F_size_w
        self.stride = stride
        self.padding = padding
        self.col = None
        
    def im2col(self, imput): 
        self.N, self.C, H, W = imput.shape 
        #(N = n_samples, H = height, W = width, C = channel_1)
        self.Nout_h = N_out_h(H, 
                              self.padding, 
                              self.F_size_h, 
                              self.stride) #問題２
        self.Nout_w = N_out_w(W, 
                              self.padding, 
                              self.F_size_w, 
                              self.stride) #問題２

        #np.pad で　imputデータ　の外堀を埋める。4次元なので4つ引数が必要。
        #順番はshape通り。N, C, H, W。
        img = np.pad(imput,[(0,0), (0,0), 
                            (self.padding,self.padding),
                            (self.padding,self.padding)])
        col = np.zeros((self.N, self.C, 
                        self.F_size_h, self.F_size_w, 
                        self.Nout_h, self.Nout_w)) 

        #strideで移動する分をまとめることでfor回数を減らす。
        for y in range(self.F_size_h):
            self.y_out = y + self.stride * self.Nout_h
            for x in range(self.F_size_w):
                self.x_out = x + self.stride * self.Nout_w
                col[:, :, y, x, :, :] = img[:, :, 
                                            y:self.y_out:self.stride, 
                                            x:self.x_out:self.stride]

        col = col.transpose(0, 4, 5, 1, 2, 3).reshape(self.N 
                                                      * self.Nout_h 
                                                      * self.Nout_w,
                                                      -1)
        #reshape(N * Nout_h * Nout_w, -1) = col after (32448000, 9)
        self.col = col
        return self.col
    
    def col2im(self, imput):
        self.N_2im, self.C_2im, H, W = imput.shape
        #入力データの形に戻す
        col =  col.reshape(self.N_2im, self.Nout_h, 
                           self.Nout_w, self.C_2im, 
                           self.F_size_h, self.F_size_w).transpose(0,3,4,5,1,2)
        #入力データを初期化
        img = np.zeros((self.N_2im, self.C_2im, 
                        H + 2 * self.padding + self.stride - 1))
        
        for y in range(self.F_size_h):
            for x in range(self.F_size_w):
                img[:,:,y:self.y_out:self.stride,
                    x:self.x_out:self.stride] = col[:,:,y,x,:,:]
        
        return img[:,:,self.padding: H + self.padding, 
                   self.padding: W + self.padding]

In [228]:
class Conv2d:
    """
    チャンネル数を1に限定しない2次元畳み込み層のクラス
    Parameters
    ----------
    channel_1 : int
      前の層のchannel数
    channel_2 : int
      後の層のchannel数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, channel_1, channel_2, padding,
                 F_size_h, F_size_w, stride, 
                 initializer, optimizer):
        
        self.optimizer = optimizer
        
        self.channel_1 = channel_1
        self.channel_2 = channel_2
        self.F_size_w = F_size_w
        self.F_size_h = F_size_h
        self.stride = stride #stride
        self.padding = padding
        
        # initializerのメソッドを使い、self.Filterとself.Bを初期化する
        self.Filter = initializer.W(channel_2, 
                                    channel_1, 
                                    self.F_size_h, 
                                    self.F_size_w)
        self.B = initializer.B(channel_2) # （出力チャンネル数）
        
        self.X = None
        self.X_array_back = np.zeros_like((self.Filter[0,:,:])) 
        #backward用
        self.X_array = None
        im_hiraki = hiraki(self.F_size_h, 
                           self.F_size_w, 
                           self.stride, 
                           self.padding)
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, channel_1, H, W)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, channel_2, Nout_h, Nout_w)
            出力
        """        
        self.X = X
        self.H_W = 0
        self.H_B = 0
        
        
        self.X_array = im_hiraki.im2col(self.X)
        # shape(N * Nout_h * Nout_w, channel_1 * F_size_h * F_size_w)
        # after reshape (32448000, 9)
        #channelが増えると axis=1 方向に次のチャネルでの行列が加わる。
        '''
        この時点では im2col でimputした" X の開き"を作成しただけ？
        重み(Filter)の掛け方は縦行列にして dot 積。
        → shape(N * Nout_h * Nout_w, 1) の形で出てくる。
        -> .transpose(N, channel, Nout_h, Nout_w)
        
        → pooling層 = MaxPool2D にぶち込む。
        ->poolingでim2col。開きを作る。
        →reshape(N, channel_2, Nout_h, Nout_w) の形にして次の畳み込み or 全結合層へ
        '''
        self.w_filters = self.Filter.flatten() # self.Filterのfaltten
        #dot積
        A = self.X_array @ self.w_filters + self.B #W(F_size),B(n_nodes2,)
        A = A.reshape(im_hiraki.N, 
                      self.channel_2,
                      im_hiraki.Nout_h,
                      im_hiraki.Nout_w)
        return A #出力 (N, channel_2, Nout_h, Nout_w)

    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, channel_2, Nout_h, Nout_w)
            出力
            = 後ろから流れてきた勾配(loss)
        Returns
        ----------        
        dX : 次の形のndarray, shape (X.shape) #4d
            前に流す勾配
        """
        self.dB = np.sum(dA,axis=1)
        """
        col2im でreshape
        dcol(col2im のimput) を4d にreshape
        loss @ self.Filter.flatten
        を　col2im で画像に戻している。
        
        ＊転置しないと数値おかしくなる？
        各チャンネルの要素和を取る　→　誤差のチャネル
        """
        dcol = dA @ self.X_array
        
        dA_imgs = im_hiraki.col2im(dcol) #col2im で 6d にreshape
        
        
        
        self.dW = loss'''loss <- 2d reshaped dA''' @ self.w_filters
        
        #dWはW.shape(フィルターの形)なので、reshape to 4d が必要。
        
#         dX = 
        
        # 更新
        self = self.optimizer.update(self) 
        return dX #4d X.shape

In [212]:
np.set_printoptions(edgeitems=3)
Con_try = Conv2d(channel_1 = 1, channel_2 = 1, padding = 0,
                 F_size_h = 3, F_size_w = 3, stride = 1, 
                 initializer = SimpleInitializer_CNN(),
                 optimizer = SGD(lr = 0.01))
Con_try.forward(X_train).shape

#forward 出力と同じ shape で簡単な数列を loss がわりにする
# Con_try.backward(dA)

(48000, 1, 26, 26)

### initializer

In [143]:
class SimpleInitializer_CNN:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, weight_type = 'sig', sigma=0.01):
        self.weight_type = weight_type
        self.sigma = sigma
    def W(self, channel_1, channel_2, 
          F_size_h, F_size_w):
        """
        重みの初期化
        #(出力チャンネル数、入力チャンネル数、フィルタサイズ)
        Parameters
        ----------
        channel_1 : int
          入力チャンネル数
        channel_2 : int
          出力チャンネル数
        F_size : int
          フィルターサイズ

        Returns
        ----------
        W :
        """
        if self.weight_type == 'sig':
            W_std = self.sigma
        
        elif self.weight_type == 'xav':
            W_std = 1/np.sqrt(channel_1)
            
        elif self.weight_type == 'he':
            W_std = np.sqrt(2/channel_1)
            
        W = W_std * np.random.randn(channel_2, channel_1, 
                                    F_size_h, F_size_w)
        return W #(出力チャンネル数、入力チャンネル数、フィルタサイズ)
    
    def B(self, channel_2):
        """
        バイアスの初期化
        Parameters
        ----------
        channel_2 : int
          出力チャンネル数

        Returns
        ----------
        B : # （出力チャンネル数）
        """
        B = self.sigma * np.random.randn(channel_2)
        return B

### optimizer

In [61]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer): 
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        '''
        layer (FC)から持ってこれるようになる。
        layer.Z = FC内のself.Z
        layer.W
        layer.B
        '''
        layer.W = layer.W - self.lr * layer.dW
        layer.B = layer.B - self.lr * layer.dB

In [45]:
class AdaGrad:
    """
    AdaGrad
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
        
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        # ミニバッチ方向(axis=0)にベクトルの平均を計算 #(n_nodes1, n_nodes2)
        layer.H_W = layer.H_W + np.sum(layer.dW ** 2)
        layer.H_B = layer.H_B + np.sum(layer.dB ** 2)
        layer.W = layer.W - (self.lr * np.sqrt(1/layer.H_W)) * layer.dW
        layer.B = layer.B - (self.lr * np.sqrt(1/layer.H_B)) * layer.dB

### activation

In [41]:
class Softmax:
    def __init__(self):
        pass    

    def forward(self,A):
        if A.ndim == 2:
            x = A.T
            x = x - np.max(x, axis=0)
            y = np.exp(x) / np.sum(np.exp(x), axis=0)
            return y.T
        
#         x = A - np.max(A) 
        # オーバーフロー対策
#         Z_last = np.exp(A) / np.sum(np.exp(A))
        return np.exp(A) / np.sum(np.exp(A))
        
#         return Z_last
    
    def backward(self,Z_last,y):
        dA_last = Z_last - y #交差エントロピー誤差の計算も含む実装
        return dA_last

In [206]:
"""
合計が１になるか確認することが大切。
"""
soft_func = Softmax()
soft = np.sum(soft_func.forward(X_train),axis=1)
# soft

/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app
/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


In [ ]:
class Sigmoid:
    def __init__(self):
        self.A = None    

    def forward(self,A):
        self.A = A
        Z = 1/ (1+ np.exp(- self.A))
        return Z
    
    def backward(self,dZ):
        dA =dZ * ((1- (1/ (1+ np.exp(- self.A))))*(1/ (1+ np.exp(- self.A))))
        return dA

In [ ]:
class Hipo:
    def __init__(self):
        self.A = None    

    def forward(self,A):
        self.A = A
        Z = np.tanh(self.A)
        return Z
    
    def backward(self,dZ):
        dA = dZ * ((np.tanh(self.A))**2)
        return dA

In [43]:
class ReLU:
    """
    活性化関数 ReLU
    """
    def __init__(self):
        self.forward_A = None
    def forward(self, A):
        self.forward_A = A
        return np.maximum(A, 0)
    def backward(self, dZ):
        return dZ * np.where(self.forward_A>0, 1, 0)

## 【問題2】2次元畳み込み後の出力サイズ
>畳み込みを行うと特徴マップのサイズが変化します。どのように変化するかは以下の数式から求められます。この計算を行う関数を作成してください。

In [68]:
def N_out_h(H, padding, F_size_h, stride):
    return (H + 2* padding - F_size_h) // stride + 1 #問題２
    
def N_out_w(W, padding, F_size_w, stride):
    return (W + 2* padding - F_size_w) // stride + 1 #問題２

## 【問題3】最大プーリング層の作成
>最大プーリング層のクラスMaxPool2Dを作成してください。プーリング層は数式で表さない方が分かりやすい部分もありますが、数式で表すとフォワードプロパゲーションは以下のようになります。

>ある範囲の中でチャンネル方向の軸は残したまま最大値を計算することになります。
バックプロパゲーションのためには、フォワードプロパゲーションのときの最大値のインデックス 
(
p
,
q
)
 を保持しておく必要があります。フォワード時に最大値を持っていた箇所にそのままの誤差を流し、そこ以外には0を入れるためです。

In [224]:
class MaxPool2D:
    def __init__(self, F_size_h = 2, F_size_w = 2,
                 stride = 2, padding = 0):
        self.max_index = None 
        self.F_size_h = F_size_h
        self.F_size_w = F_size_w
        self.stride = stride
        self.padding = padding
    
    def forward(self, A):
        im_hiraki = hiraki(self.F_size_h, 
                           self.F_size_w, 
                           self.stride, 
                           self.padding)
        A_array = im_hiraki.im2col(A)
        #shape (A_sumples *N_in_h/F_size_h *N_in_w/F_size_w, C*F_size_h*F_size_w)
        self.A_max = np.argmax(A_array, axis = 1) #index
        #shape (N *N_in_h/F_size_h *N_in_w/F_size_w,)
        '''
        最大を取ってくる。また、最大値のインデックス (p,q)をforwardからbackに渡す。
        A_max_array = A_max.reshape()
        '''
        return A_max_array  #shape(N, C, N_in_h/F_size_h, N_in_w/F_size_w)
    
    def backward(self, dX):
        '''max のみ返す。他は0で返す'''
        #hirakiにする　→　np.zeros_like()[self.A_max]=A_array[self.A_max] ?
        
        return dA

In [222]:
maxpoo = MaxPool2D()
maxpoo.forward(Con_try.forward(X_train)).shape

(8112000, 4)
(8112000,)


## 【問題4】（アドバンス課題）平均プーリングの作成
>平均プーリング層のクラスAveragePool2Dを作成してください。
範囲内の最大値ではなく、平均値を出力とするプーリング層です。
画像認識関係では最大プーリング層が一般的で、平均プーリングはあまり使われません。

## 【問題5】平滑化
>平滑化するためのFlattenクラスを作成してください。
フォワードのときはチャンネル、高さ、幅の3次元を1次元にreshapeします。その値は記録しておき、バックワードのときに再びreshapeによって形を戻します。この平滑化のクラスを挟むことで出力前の全結合層に適した配列を作ることができます。



In [234]:
class Flatten:
    def __init__(self):
        self.shape = None
    
    def forward(self, Z):
        self.shape = Z.shape
        #batch size 残す。　
        Z.reshape(-1)
        return Z　#２d
    
    def backward(self, dA):
        return dA.reshape(self.shape)

## 【問題6】学習と推定
>作成したConv2dを使用してMNISTを学習・推定し、Accuracyを計算してください。



In [235]:
class Scratch2dCNNClassifier():
    """
    CNN 分類器

    Parameters
    ------    

    Attributes
    ----------
    """
    def __init__(self, max_iter=5,
                 lr=0.01, sigma = 0.01,
                 verbose = True):
        """
        self.sigma : ガウス分布の標準偏差
        self.lr : 学習率
        self.iter : 学習回数
        """
        self.iter = max_iter
        self.lr = lr
        self.sigma = sigma
        self.verbose = verbose
    
    def loss_func(self,y,y_pred_proba):
        sigma_c = np.sum(y * np.log(y_pred_proba), axis = 1) 
        #batch_size方向に平均
        return -np.mean(sigma_c)
    
    def fit(self, X, y,
            sigma = 0.01,
            n_output = 10):
        """
        ニューラルネットワーク分類器を学習する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練用データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練用データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証用データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証用データの正解値
        
        # self.n_nodes1 : 1層目のノード数
        # self.n_nodes2 : 2層目のノード数
        # self.n_output : 出力層のノード数
        """
        self.n_features = X.shape[1]
        self.n_output = n_output
        self.channel_1 = 1
        self.channel_2 = 1
        
        optimizer = SGD(self.lr) #更新方法選択
        self.Conv1 = Conv2d(channel_1 = 1, channel_2 = 1, padding = 0,
                            F_size_h = 3, F_size_w = 3, stride = 1, 
                            initializer = SimpleInitializer_CNN(),
                            optimizer = optimizer)
        self.activation1= ReLU()
        
        self.Conv2 = Conv2d(channel_1 = 1, channel_2 = 1, padding = 0,
                            F_size_h = 3, F_size_w = 3, stride = 1, 
                            initializer = SimpleInitializer_CNN(),
                            optimizer = optimizer)
        self.activation2= ReLU()
        
        
        self.FC1 = FC(n_nodes1 = self.channel_1,
                      n_nodes2 = self.n_output,
                      initializer = SimpleInitializer_fullyconnect(weight_type='sig', sigma = self.sigma), 
                      optimizer = SGD(self.lr))
        self.activation3 = Softmax()
        
        self.flatten = Flatten()
        
        self.loss = None
        
       
        
        
        
        for n in range(self.iter): #何回回そう？ MNISTなら50回ほどで試す
            get_mini_batch = GetMiniBatch(X_train,
                                          y_train_one_hot,
                                          batch_size = 20)
            for mini_X_train, mini_y_train in get_mini_batch:
            # このfor文内でミニバッチが使える
                #W,B = FC内 FCX.W, FCX.B (Xは1,2,...)
                A1 = self.Conv1.forward(mini_X_train) 
                Z1 = self.activation1.forward(A1) 
                A2 = self.Conv2.forward(Z1)
                Z2 = self.activation2.forward(A1) 
                """
                ノード　＝　チャンネル数　✖️　チャンネルの要素数
                出チャンネル数を　（平滑化）　
                
                conv
                -> pool
                -> activate
                
                -> conv
                ...
                -> activate
                
                -> flatten
                -> FC
                -> activate
                """
                Z2 = self.flatten.forward(Z2)
                A3 = self.FC1.forward(Z2) #(batch_size, n_nodes1)
                Z3 = self.activation3.forward(A3) #(batch_size, n_nodes1)                
                
                dA3 = self.activation3.backward(Z2, mini_y_train) #(batch_size, n_output)
                # 交差エントロピー誤差とソフトマックスを合わせている
                dX2 = self.FC1.backward(dA3) #(batch_size, n_nodes1)
                dA2 = self.activation2.backward(dX2)
                dA2 = self.fratten.backward(dA2)
                dX1 = self.Conv2.backward(dA2)
                dA1 = self.activation1.backward(dX1) 
                dX0 = self.Conv1.backward(dA1) # dZ0は使用しない
            
            A1 = self.Conv1.forward(X) 
            Z1 = self.activation1.forward(A1) 
            A2 = self.Conv2.forward(Z1)
            Z2 = self.activation2.forward(A1) 
            Z2 = self.flatten(Z2)
            A3 = self.FC1.forward(Z2) #(batch_size, n_nodes1)
            Z3 = self.activation3.forward(A3)     
            y_pred_proba = Z3
            loss = self.loss_func(y,y_pred_proba)
            self.loss = np.append(self.loss,loss)
            
        self.loss = np.delete(self.loss, 0)
        if self.verbose:
            #verboseをTrueにした際は学習過程などを出力する
            print("self.loss.shape",self.loss.shape)
                
    def predict(self, X):
        """
        ニューラルネットワーク分類器を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            推定結果
        """
        pass
    
        return #y_pred_proba

In [236]:
CNN2d = Scratch2dCNNClassifier(max_iter=1,
                               lr=0.001, sigma = 0.01,
                               verbose = True)

In [237]:
y_train_one_hot.shape #(48000, 10)

(48000, 10)

In [ ]:
t1 = time.time()
CNN2d.fit(X_train, y_train_one_hot)

t2 = time.time()
print("processing time:",t2-t1)

## 【問題7】（アドバンス課題）LeNet
>CNNで画像認識を行う際は、フィルタサイズや層の数などを１から考えるのではなく、有名な構造を利用することが一般的です。現在では実用的に使われることはありませんが、歴史的に重要なのは1998年の LeNet です。この構造を再現してMNISTに対して動かし、Accuracyを計算してください。



## 【問題8】（アドバンス課題）有名な画像認識モデルの調査
>CNNの代表的な構造としてははAlexNet(2012)、VGG16(2014)などがあります。こういったものはフレームワークで既に用意されていることも多いです。
どういったものがあるか簡単に調べてまとめてください。名前だけでも見ておくと良いでしょう。

https://keras.io/ja/applications/

## 【問題9】出力サイズとパラメータ数の計算
>CNNモデルを構築する際には、全結合層に入力する段階で特徴量がいくつになっているかを事前に計算する必要があります。
また、巨大なモデルを扱うようになると、メモリや計算速度の関係でパラメータ数の計算は必須になってきます。フレームワークでは各層のパラメータ数を表示させることが可能ですが、意味を理解していなくては適切な調整が行えません。

## 【問題10】（アドバンス課題）フィルタサイズに関する調査
>畳み込み層にはフィルタサイズというハイパーパラメータがありますが、2次元畳み込み層において現在では3×3と1×1の使用が大半です。以下のそれぞれを調べたり、自分なりに考えて説明してください。
- 7×7などの大きめのものではなく、3×3のフィルタが一般的に使われる理由
- 高さや幅方向を持たない1×1のフィルタの効果
